<a href="https://colab.research.google.com/github/hanhluukim/replication-topic-modelling-in-embedding-space/blob/main/BERT_Embedding_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers

     |████████████████████████████████| 4.2 MB 8.2 MB/s 
     |████████████████████████████████| 84 kB 3.6 MB/s 
     |████████████████████████████████| 596 kB 18.6 MB/s 
     |████████████████████████████████| 6.6 MB 44.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
!pip install stop-words

  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32911 sha256=7e2febffa6082f77f90ac03536c8bf5382d6a007fd03e03730e0e13843564fa7
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
Successfully built stop-words


In [4]:
# using bert-model to create static the word-embedding
# https://datascience.stackexchange.com/questions/85566/how-pre-trained-bert-model-generates-word-embeddings-for-out-of-vocabulary-words

from distutils.util import split_quoted
import torch
from transformers import BertTokenizer, BertModel
from sklearn.datasets import fetch_20newsgroups
import re
from nltk.tokenize import word_tokenize
import nltk
import string
nltk.download('punkt')

#with open('src/stops.txt', 'r') as f:
#    stops = f.read().split('\n')

from stop_words import get_stop_words
stop_words = get_stop_words('en')
    

newsgroups_train = fetch_20newsgroups(subset='train')

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )
model.eval()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


OSError: ignored

In [ ]:
# read train_data from 20newsgroups
def read_raw_documents():
    raw_documents = []
    raw_labels = []
    for i in range(0,len(newsgroups_train.data)):
        raw_documents.append(newsgroups_train.data[i])
        raw_labels.append(newsgroups_train.target[i])
    return raw_documents, raw_labels

def simple_preprocess(raw_documents):
    def only_letters(tested_string):
        for letter in tested_string:
            if letter not in "abcdefghijklmnopqrstuvwxyz":
                return False
        return True
    def clean_doc_for_bert(doc): 
        doc = doc.replace(">","").lower()
        word_list = word_tokenize(doc) #only using empty space and punctation for tokenization
        cleaned = []
        for w in word_list:
            if w not in stop_words:
                if w in string.punctuation or only_letters(w): #using only character from punctation and alpha characters
                    if w in string.punctuation or len( set(w) ) > 1: #punctation with len 1 allowed but alpha word must be longer then 1
                        cleaned.append( w)
        return " ".join(cleaned), cleaned  #save doc in string and in token-list         
       
    cleaned_documents = []
    for doc in raw_documents:
        doc_in_string, doc_in_token_list = clean_doc_for_bert(doc)
        cleaned_documents.append(doc_in_string)
    return cleaned_documents

def transform_to_sentences_with_labels():
    # we will not use labels
    sentences_with_labels = []
    return sentences_with_labels

def fine_tune_bert():
    # should to be trained?
    # no, because for topic modelling, that is usupervised problem. We just find topics for the documents
    # topic modelling no targets
    return True

def transform_to_sentences(docs): #no labels
    data_as_sentences = []
    for doc in docs:
      for sent in doc.split(". "): #make sentences
        updated_sent = " ".join([t for t in sent.split(" ") if len(t) > 1])
        if len(updated_sent.split(" ")) > 1:
            data_as_sentences.append(updated_sent)
        else:
            if updated_sent not in data_as_sentences:
                data_as_sentences.append(updated_sent)
    return data_as_sentences

def split_long_sentence(splitted_sent, given_len):
    subsents = []
   #for i in range(0,len(splitted_sent), given_len):
    i=0
    while i < len(splitted_sent): 
        if i == 0:
            sub = " ".join(splitted_sent[i:i+given_len])
            subsents.append(sub)
            i = i + given_len
        if i!=0:
            j = i + given_len - 5 #windown 5
            if j + given_len <= len(splitted_sent):
                sub = " ".join(splitted_sent[j:j + given_len])
                subsents.append(sub)
            else:
                sub = " ".join(splitted_sent[j:])
                if len(sub)>1:
                    subsents.append(sub)
            i = j + given_len
    return subsents

def handle_long_sentences(sentences, given_len):
    # overlapped splitting sentence windown 5
    subsents = []
    deleted_long_sents = []
    for sent in sentences:
        splitted_sent = sent.split(" ")
        if len(splitted_sent) > given_len:
          long_sent_subsents = split_long_sentence(splitted_sent, given_len)
          subsents.extend(long_sent_subsents)
          deleted_long_sents.append(sent)
    # update sentences: remove and add subsents
    for del_sent in deleted_long_sents:
        sentences.remove(del_sent)
    for add_sent in subsents:
        sentences.append(add_sent)
    return sentences

def create_marked_senteces(sentences):
    return ['[CLS] ' + sent.strip() + ' [SEP]' for sent in sentences]
def save_sents_to_txt(shorted_sentences):
    with open(r'./bert_sentences.txt', 'w') as fp:
      for sent in shorted_sentences:
          # write each item on a new line
          fp.write(f'{sent} \n')
      print('saving sentences from bert-processing')
    return True

# **Datenvorbereitung**

In [ ]:
print("reading data:...")
raw_documents, _ = read_raw_documents()
print(len(raw_documents))
print("preprocess data:...")
preprocessed_docs = simple_preprocess(raw_documents)
print(len(preprocessed_docs))
print("transform to sentences:...")
sentences = transform_to_sentences(preprocessed_docs)
print("split sentences to 128 tokens:...")
shorted_sentences =  handle_long_sentences(sentences, 128)
shorted_sentences = create_marked_senteces(shorted_sentences)
# write sentences to txt files
with open(r'bert_sentences.txt', 'w') as fp:
    for sent in shorted_sentences:
        # write each item on a new line
        fp.write(f'{sent} \n')
    print('saving sentences from bert-processing')
print("finished: ...")



# **Embeddings**

In [ ]:
from transformers import BertTokenizerFast, BertTokenizer
tokenizer_ = BertTokenizer.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )
model.eval()

# **Information nach dem Tokenizer**

In [ ]:
def tokenizer_for_a_sent(sent, tokenizer):
  this_sent_tokenizer = tokenizer(sent)
  # index of token in the vocabulary
  indexed_tokens = this_sent_tokenizer.input_ids
  segments_ids = [1] * len(indexed_tokens)
  # Convert inputs to PyTorch tensors
  tokens_tensor = torch.tensor([indexed_tokens])
  segments_tensors = torch.tensor([segments_ids])
  return tokens_tensor, segments_tensors

In [ ]:
def reform_token_embeddings_of_sentence(full_outputs):
  hidden_states = outputs[2]
  token_embeddings = torch.stack(hidden_states, dim=0)
  token_embeddings = torch.squeeze(token_embeddings, dim=1) # size= (n_hidden_layers, n_tokens, 768)
  token_embeddings = token_embeddings.permute(1,0,2) # size= (n_tokens, n_hidden_layers, 768)
  return token_embeddings 

def get_token_embeddings(reformed_token_embeddings):
  # using sum four last layers
  token_vecs_sum = []
  for token in token_embeddings: 
    sum_vec = torch.sum(token[-4:], dim=0)
    token_vecs_sum.append(sum_vec)
    print ('Shape is: %d x %d' % (len(token_vecs_sum), len(token_vecs_sum[0])))
  return token_vecs_sum # size: n_tokens: 768

In [ ]:
"""
def find_belonged_embeddings_of_token(token, tokenized_text, embeddings_outputs):
  belonged_embeddings = []
  belonged_indices = []
  for idx, t in enumerate(tokenized_text):
    if t.strip("#") in token:
      belonged_indices.append(i)
  for idx in belonged_indices:
    belonged_embeddings.append(embeddings_outputs[i])
  return belonged_embeddings
"""
def get_belonging_embeddings_of_word(word_id, tokenized_indices, tokens_embeddings):
  belongging_embeddings_of_word = []
  for idx, tokenizer_idx in enumerate(tokenized_indices):
    if tokenizer_idx == word_id:
      belongging_embeddings_of_word.append(tokens_embeddings[idx])
  return belongging_embeddings_of_word

def get_word_embedding(belonging_embeddings=None, methode="mean"):
  return torch.mean(belonging_embeddings, dim=0)

def get_all_word_embeddings_in_sent(original_sent, sent_tokens_ids, sent_outputs_tokens_embeddings):
    # todo what is one word in multiple position in sentence
    # remove the special token ids from sent_tokens_ids
    def need_to_update(sent_tokens_ids):
      return sent_tokens_ids
    sent_tokens_id = need_to_update(sent_tokens_ids)
    splitted_words_of_sent = original_sent.split(" ")
    # make ids constitent 
    sent_words_ids = { i+1:word for i, word in enumerate(splitted_words_of_sent)}
    sent_words_embeddings = {}
    unique_tokens_ids_list = list(set(sent_tokens_ids)) #neead remove the id of special tokens, which are not in the original sentence
    if len(sent_words_ids.keys()) == len(unique_tokens_ids_list):
      for unique_token_id in unique_tokens_ids_list:
        original_word = sent_words_ids[unique_token_id]
        word_belonging_embeddings = get_belonging_embeddings_of_word(unique_token_id, sent_tokens_ids, sent_outputs_tokens_embeddings)
        sent_words_embeddings[original_word] = get_word_embedding(word_belonging_embeddings)
      return sent_words_embeddings
    else:
      return False
"""
def get_word_embeddings_from_all_senteneces_for_a_word(vocab, data_sentences_in_embeddings):
  sum_embeddings = None
  counts = None
  for word in vocab:
    for i in range(data_sentences_in_embeddings):
      embeddings_in_sent = data_sentences_in_embeddings[i]


  return torch.mean(word_embeddings_from_all_senteneces)
"""

In [ ]:
def embeddings_using_tokenizer():
  return True
def embeddings_using_tokenizerfast():
  return True

In [ ]:
ex_sent = shorted_sentences[0]
print(ex_sent)

# using BertTokenizer
tokens = tokenizer_.tokenize(ex_sent)
print(tokens)
print(len(tokens))
indexed_tokens = tokenizer_.convert_tokens_to_ids(tokens)
print(indexed_tokens)
segments_ids = [1] * len(tokens)
print(segments_ids)
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])
with torch.no_grad():
    outputs = model(tokens_tensor, segments_tensors)
    hidden_states = outputs[2]
    print(len(hidden_states))
    token_embeddings = torch.stack(hidden_states, dim=0)
    print(token_embeddings.size())
    token_embeddings = torch.squeeze(token_embeddings, dim=1)
    print(token_embeddings.size())
    #print(hidden_states.shape)
# using BertTokenizerFast
    reformed = reform_token_embeddings_of_sentence(outputs)
    sent_tokens_embeddings = get_token_embeddings(reformed)

In [ ]:
from transformers import BertTokenizerFast

tokenizerfast = BertTokenizerFast.from_pretrained('bert-base-uncased')
#tokens = t('word embeddings are vectors', add_special_tokens=False, return_attention_mask=False, return_token_type_ids=False)
#print(tokens.word_ids())
tokenized_text = tokenizerfast(sent)
# index of token in the vocabulary
indexed_tokens = tokenized_text.input_ids

segments_ids = [1] * len(tokenized_text)
# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])
#print(tokenized_text.input_ids)
#print(tokenized_text.token_type_ids)
#print(tokenized_text.word_ids())

In [ ]:
for i in range(0,len(sentences)):
  sent = shorted_sentences[i]
  tokens_tensor, segments_tensors = tokenizer_for_a_sent(sent)
  """
  tk = tokenizer(sent)
  print(tk.input_ids)
  print(tk.token_type_ids)
  print(tk.word_ids())
  print(len(tk.word_ids()))
  print(tokenizer.tokenize(sent))
  print(len(tokenizer.tokenize(sent)))
  print("---------------------------------------")
  """
  with torch.no_grad():
    outputs = model(tokens_tensor, segments_tensors)
    hidden_states = outputs[2]
    #print(hidden_states)
    print(len(hidden_states))
    print(hidden_states[0].shape)